In [ ]:
import csv
from time import sleep
from selenium.webdriver.common.keys import Keys
import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select



options = selenium.webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = selenium.webdriver.Chrome('drivers/chromedriver')

driver.get('https://twitter.com/login')

In [ ]:
sleep(2.5)
username = driver.find_element(by=By.XPATH, value='//input[@autocomplete="username"]')
username.send_keys('zat.tyme.su@gmail.com')
username.send_keys(Keys.RETURN)


def write_password(driver):
    sleep(1.5)
    password = driver.find_element(by=By.XPATH,
                                   value='//input[@autocomplete="current-password"]')
    password.send_keys('Twitter2022$')
    password.send_keys(Keys.RETURN)


In [ ]:
try:
    write_password(driver)
    sleep(2)


except:
    login = driver.find_element(by=By.XPATH,
                                value='//input[@autocomplete="on"]')
    login.send_keys('ferkin_kz')
    login.send_keys(Keys.RETURN)
    sleep(1)

    write_password(driver)
    sleep(2)

In [ ]:
driver.get('https://twitter.com/elonmusk')
sleep(4)

In [ ]:
posts = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

In [ ]:
post = posts[0]


In [178]:
post = posts[0]


'Elon Musk Retweeted\nSpaceX\n@SpaceX\n·\n7h\nVideo of Dragon’s Draco thrusters moving the spacecraft closer to the \n@Space_Station\n0:07\n2.1M views\n1,674\n4,291\n39.2K'